In [2]:
from osgeo import gdal
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt


Matplotlib is building the font cache; this may take a moment.


In [20]:
print(os.getcwd())
os.chdir('/Volumes/Kew Back Up 1/KEW/Cluster_Code/March_25_Run/June_Output/Processing')
print(os.getcwd())
df = pd.read_csv('Processed_Output_minCD_numanalog.csv')
print(df.head())

/Volumes/Kew Back Up 1/KEW/Cluster_Code/March_25_Run/June_Output/processing
/Volumes/Kew Back Up 1/KEW/Cluster_Code/March_25_Run/June_Output/processing
           x          y  min_distance  num_distances
0 -44.779306  59.995694         0.000              1
1 -33.887639  83.654027        17.780              3
2 -33.787639  83.654027        38.399              1
3 -33.754306  83.654027        32.247              1
4 -31.345973  83.654027        16.413              4


In [21]:

df_noNA = df.dropna(subset=["min_distance"])
df_noNA.to_csv("Processed_Output_minCD_numanalog_clean.csv", index=False)

In [23]:

ds = gdal.Open("/Volumes/Kew Back Up 1/KEW/Cluster_Code/Climate_inputs/T1_temp_full.tif")
ar = ds.GetRasterBand(1).ReadAsArray()
flat = ar.flatten()
gt = ds.GetGeoTransform()
res = gt[1]
xmin = gt[0]
ymax = gt[3]
xsize = ds.RasterXSize
ysize = ds.RasterYSize
xstart = xmin +res/2
ystart = ymax - res/2
ds = None
noDataValue = -9999

In [25]:
with open("OUTPUT_minCD.vrt", "w") as f:
    f.write('''<OGRVRTDataSource>
    <OGRVRTLayer name="Processed_Output_minCD_numanalog_clean"> 
        <SrcDataSource relativeToVRT="1">/Volumes/Kew Back Up 1/KEW/Cluster_Code/March_25_Run/June_Output/Processing/Processed_Output_minCD_numanalog_clean.csv</SrcDataSource>
        <LayerSRS>WGS84</LayerSRS>
        <GeometryType>wkbPoint</GeometryType>
        <GeometryField encoding="PointFromColumns" x="x" y="y" z="min_distance"/>
    </OGRVRTLayer>
</OGRVRTDataSource>''')


In [26]:
r = gdal.Rasterize("OUTPUT_minCD.tif", "OUTPUT_minCD.vrt", outputSRS = "EPSG:4326", 
                          xRes = res, yRes = res, attribute = "min_distance", noData = noDataValue)
r=None

In [18]:
with open("OUTPUT_numanalog.vrt", "w") as f:
    f.write('''<OGRVRTDataSource>
    <OGRVRTLayer name="Processed_Output_minCD_numanalog"> 
        <SrcDataSource relativeToVRT="1">/Volumes/Kew Back Up 1/KEW/Cluster_Code/March_25_Run/June_Output/Processing/Processed_Output_minCD_numanalog.csv</SrcDataSource>
        <LayerSRS>WGS84</LayerSRS>
        <GeometryType>wkbPoint</GeometryType>
        <GeometryField encoding="PointFromColumns" x="x" y="y" z="num_distances"/>
    </OGRVRTLayer>
</OGRVRTDataSource>''')

In [19]:
r = gdal.Rasterize("OUTPUT_numanalog.tif", "OUTPUT_numanalog.vrt", outputSRS = "EPSG:4326", 
                          xRes = res, yRes = res, attribute = "num_distances", noData = noDataValue)
r=None